# Challenge 3

In this challenge we will work on the `Orders.csv` data set in the previous [Subsetting and Descriptive Stats lab](../../lab-subsetting-and-descriptive-stats/your-code/main.ipynb). In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

# Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [41]:
# import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


Next, import `Orders.csv` from the "subsetting" lab folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [42]:
# enter your code here
orders = pd.read_csv("Orders.csv")

orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [43]:
# your code here
# Aggregate amount_spent for unique customers
sum_by_customer = orders.loc[:, ['CustomerID', 'amount_spent']].groupby('CustomerID').sum()


perc_95 = np.percentile(sum_by_customer, 95)
perc_75 = np.percentile(sum_by_customer, 75)
print(perc_95, perc_75)
sum_by_customer


5840.181999999982 1661.64


,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40
...,...
18280,180.60
18281,80.82
18282,178.05


In [44]:
# customer by quantile range
vip = sum_by_customer[sum_by_customer.amount_spent > perc_95]
vip


,amount_spent
CustomerID,
12346,77183.60
12357,6207.67
12359,6372.58
12409,11072.67
12415,124914.53
...,...
18109,8052.97
18139,8438.34
18172,7561.68


In [45]:
preferred = sum_by_customer[(sum_by_customer.amount_spent < perc_95) & (sum_by_customer.amount_spent > perc_75)]
preferred

,amount_spent
CustomerID,
12347,4310.00
12348,1797.24
12349,1757.55
12352,2506.04
12356,2811.43
...,...
18259,2338.60
18260,2643.20
18272,3078.58


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

# Q2: How to identify which country has the most VIP Customers?

# Q3: How to identify which country has the most VIP+Preferred Customers combined?

Provide your solution for Q2 below:

In [46]:
orders2 = orders.copy()
orders2.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [40]:
preferred_customers = preferred.index
vip_customers = vip.index


In [47]:
# your code here
orders2['Status'] =  np.where(orders['CustomerID'].isin(vip_customers),'VIP',
                        np.where(orders['CustomerID'].isin(preferred_customers),'Preferred', 'Regular'))

In [59]:
orders2.loc[orders2['Status'].value_counts()]

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,Status
151781,217093,555884,85099B,2011,6,2,15,jumbo bag red retrospot,10,2011-06-07 15:44:00,1.79,17389,United Kingdom,17.90,VIP
141659,204166,554638,21932,2011,5,3,11,scandinavian paisley picnic bag,2,2011-05-25 11:59:00,1.65,14810,United Kingdom,3.30,Preferred
104484,154160,549851,22037,2011,4,2,15,robot birthday card,12,2011-04-12 15:08:00,0.42,16062,United Kingdom,5.04,Regular


In [88]:
orders_vip = orders2[orders2['Status'] == 'VIP']

orders_vip.groupby('Country').agg({'Status':'count'}).sort_values('Status', ascending=False)

,Status
Country,
United Kingdom,84185
EIRE,7077
France,3290
Germany,3127
Netherlands,2080
Australia,898
Portugal,681
Switzerland,594
Spain,511


In [91]:
orders_vip_preferred = orders2[orders2['Status'].isin(['VIP', 'Preferred'])]
orders_vip_preferred.groupby('Country').agg({'Status':'count'}).sort_values('Status', ascending=False)

,Status
Country,
United Kingdom,221635
Germany,7349
EIRE,7238
France,6301
Netherlands,2080
Spain,1569
Belgium,1557
Switzerland,1370
Portugal,1093


In [83]:
orders2.groupby(['Country','Status']).agg({'Status':'count'})

Status
Country              Status           
Australia            Preferred     130
                     Regular       157
                     VIP           898
Austria              Preferred     158
                     Regular       240
...                                ...
United Arab Emirates Regular        68
United Kingdom       Preferred  137450
                     Regular    132710
                     VIP         84185
Unspecified          Regular       244

[76 rows x 1 columns]

In [72]:
orders2['Status'].value_counts()

Preferred    151781
Regular      141659
VIP          104484
Name: Status, dtype: int64

In [79]:
pivote = pd.pivot_table(orders2, values='CustomerID', 
            columns='Status', 
            index='Country', 
            aggfunc=({'CustomerID':'count'}))
pivote

Status,Preferred,Regular,VIP
Country,,,
Australia,130.0,157.0,898.0
Austria,158.0,240.0,NaN
Bahrain,NaN,17.0,NaN
Belgium,1503.0,474.0,54.0
Brazil,NaN,32.0,NaN
Canada,135.0,16.0,NaN
Channel Islands,225.0,159.0,364.0
Cyprus,203.0,163.0,248.0
Czech Republic,NaN,25.0,NaN
